# Sampling and the Central Limit Theorem

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import requests
from scipy import stats

%matplotlib inline

## Agenda 

SWBAT:
- Differentiate between population and sample;
- Define and calculate standard error;
- Use `numpy` to randomly sample a distribution;
- Describe the Central Limit Theorem and connect it to our knowledge of distributions and sampling.

## Descriptive vs Inferential Statistics

- Descriptive Statistics
   > Simply describe what is observed. The average height of a player on a high school football team can be directly calculated by measuring all of the current players' heights.
  
- Inferential statistics 
    > Say something general about a larger group of subjects than those we have measured. For example, we would be doing inferential statistics if we used the information about our team above to say something about the average height of *all* high school football players.

    - To put it another way, statistical inference is the process by which we take observations of a subset of a group and generalize to the whole group.

## Population Inference

The mayor's office has hired Flatiron Data Science Immersive students to determine a way to fix traffic congestion. A good starting point is to determine what proportion of the population of Seattle owns a car.

In order for us to make any determinations about a population, we must first get information about it.

Because it's usually completely impractical to get data about *everyone* in a population, we must take a sample.

### Key Terms

 - the entire group is known as the **population**  
 - the subset is a known as the **sample**

![pop](./img/sample_pop.png)

- We would use samples if the population is:
    - Too big to enumerate
    - too difficult/time consuming or expensive to sample in its entirety.

**Random sampling is not easy to do**  
Continuing our Seattle car example, how would we take a sample? 

Here are two strategies we might employ:

* Stand outside of Flatiron at 12 pm and ask random people until *n* responses


* Go to a randomly assigned street corner and at a random time and ask *n* people if they own a car

Which strikes you as better?

What do we want our sample to look like?

In particular, what relationship do we want between the sample and the population? What steps can we take to improve our odds of success in achieving this?

## Population vs. Sample Terminology

Characteristics of populations are called **parameters**

Characteristics of a sample are called **statistics**

A sample statistic is a **point estimate** of the population parameter

![imgsample](./img/sample_stats.png)

## A Simulation to Reinforce Our Definitions

Let's create a population of systolic blood pressure of adult males in Chicago, assuming a mean of 114 mmHg with a standard deviation of 11 mmHg.  We will also assume the adult male population to be 1.5 million. 

It is impossible to measure the systolic blood pressure of every man in Chicago, but let's assume multiple investigations have led to the conclusion the the mean and std of this population is 114 and 11, respecively. These are therefore estimators of the population parameter.

$\Large\hat\mu = 114$  
$\Large\hat\sigma = 11$



In [ ]:
pop = int(1.5*10**6)

# Use numpy to generate a normal distribution of the 
sys_pop = np.random.normal(loc=114, scale=11, size=pop)

fig, ax = plt.subplots()

sns.kdeplot(sys_pop, ax=ax, shade=True)
ax.set_title('Distribution of Adult Male Systolic Blood Pressure')
ax.set_xlabel('Systolic BP');

Let's then imagine we develop an effective manner of random sampling, and simulate with numpy. Our sample size is 40 people.


In [ ]:
sample_size = 40
sample = np.random.choice(sys_pop, sample_size)

# We can look at the distribution of the values in the sample.

In [ ]:
fig, ax = plt.subplots()
sns.distplot(sample, ax=ax, bins=15)
ax.set_title('Sample Distribution of Systolic BP Measurements');

We can then calculate the sample statistics:

In [ ]:
print(f'Sample mean: {sample.mean()}')
print(f'Sample standard deviation: {sample.std()}')
print(f'Sample median: {np.median(sample)}')

If we repeated this process, taking samples of the population repeatedly, we would get an array of sample statistics.

In [ ]:
number_of_samples = 1000
sample_size = 50
sample_stats = []

for _ in range(number_of_samples):
    sample = np.random.choice(sys_pop, sample_size)
    # collect the mean of each of the 1000 samples in sample stats
    sample_stats.append(sample.mean())

The collection of sample stats represents our __sampling distribution__

In [ ]:
fig, ax = plt.subplots()
ax.hist(sorted(sample_stats), bins=20)
ax.set_title('Sampling Distribution\n of Systolic BP')
ax.set_xlabel("Systolic Blood Pressure")
ax.set_ylabel('Count');

An interesting property of this sampling distribution:
    
As we continue to sample, the mean of the sampling distribution gets closer and closer to the population mean.

### Standard Error of the Mean

The standard error of the mean is the standard deviation of the sampling distribution.
The issue is that a sample is not an exact replica of the population. We need to account for that fact in order to make our estimate of the $\mu$ value possible. Let's break it down:

**Population sigma** <br/>

$\large\sigma _{x} = \frac{\sigma }{\sqrt{n}}$

* $ \sigma _{x}$ = standard error of $\bar{x} $
* $ \sigma $ = standard deviation of population

**What if we do not know the population sigma?**<br>
If we do not know the population standard deviation, we can approximate it by using the sample standard deviation.

$\large\sigma _{x} ≈ \frac{s}{\sqrt{n}}$

* s = sample standard deviation

**Sample size impact on standard error of mean**<br>

How should sample size influence standard error of the mean?

It will get *smaller* as sample size *increases*

![error](./img/diminishing_error.png)  
Important implication: The Standard Error of the mean remains the same as long as the population standard deviation is known and sample size remains the same.


In [ ]:
def standard_error(distribution, largest_sample_size, population_std=None):
    
    '''
    Calculate the standard errors for a range of sample sizes
    to demonstrate how standard error decreases when sample 
    size increases.
    '''
 
    std_errors = {}
    
    for sample_size in range(50, largest_sample_size+1):
        sample = np.random.choice(distribution, size=sample_size, replace=True)
        # Standard error with sample distribution standard deviation 
        # in place of population
        if population_std == None:
            std_err = np.std(sample)/np.sqrt(sample_size)
            std_errors[sample_size] = std_err
        
        else:
            std_err = population_std/np.sqrt(sample_size)
            std_errors[sample_size] = std_err
        
    return std_errors

In [ ]:
std_errors = standard_error(sys_pop, 1000)

fig, ax = plt.subplots()

sns.scatterplot(list(std_errors.keys()), list(std_errors.values()));

In [ ]:
std_errors = standard_error(sys_pop, 1000, population_std=114)

fig, ax = plt.subplots()

sns.scatterplot(list(std_errors.keys()), list(std_errors.values()));

## Central Limit Theorem

If we take repeated samples of a population, the sampling distribution of sample means will approximate to a normal distribution, no matter the underlying distribution!

## $E(\bar{x_{n}}) = \mu$

as n --> "large"

[good D3 example](https://seeing-theory.brown.edu/probability-distributions/index.html)

[good video demonstration](https://www.youtube.com/watch?v=jvoxEYmQHNM)


Let's look at an example taken from the ubiquitous Iris dataset. This histogram represents the distributions of sepal length:


![probgif](./img/probability-basics.gif)

https://www.kaggle.com/tentotheminus9/central-limit-theorem-animation

As we will see in hypothesis testing, pairing this theorem with the Empirical rule will be very powerful.

![empirical](img/empirical_rule.png)



Knowing that any sampling distribtion, no matter the underlying population distribution, will approach normality, we will be able to judge, given the empirical rule, how rare a given sample statistic is.  

### Bike Example
Capital bike share is trying to figure out their pricing for members versus non-members. The first step in their analysis is to see if members vs non-members ride for different amounts of time per ride.

Let's head over [here](https://s3.amazonaws.com/capitalbikeshare-data/index.html) for some DC bike data!

In [ ]:
df = pd.read_csv('/Users/gdamico/Downloads/202101-capitalbikeshare-tripdata.csv')

In [ ]:
df.tail()

### Let's take a look at the shape of our dataset

In [ ]:
df['Duration'] = pd.to_datetime(df['ended_at']) - pd.to_datetime(df['started_at'])

In [ ]:
df['Duration'] = df['Duration'].dt.total_seconds()

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 10))
ax[0].hist(df.Duration)
sns.kdeplot(df.Duration, ax=ax[1])
sns.boxplot(df.Duration, ax=ax[2]);

The shape is difficult to see because of the outliers. Let's remove some to get a better sense of the shape:

#### Removing outliers

In [ ]:
df[df['Duration'] < 0].shape[0]

df = df[df['Duration'] >= 0]

In [ ]:
pop_no_outliers = df[np.abs(stats.zscore(df.Duration)) < 3]

fig, ax = plt.subplots()
sns.distplot(pop_no_outliers.Duration);

In [ ]:
member_df = df[df['member_casual'] == 'member']
casual_df = df[df['member_casual'] == 'casual']

In [ ]:
plt.boxplot(member_df['Duration']);

In [ ]:
print(member_df.shape)
sum(abs(stats.zscore(member_df.Duration)) > 3)

In [ ]:
member_df_nooutliers = member_df[np.abs(stats.zscore(member_df.Duration)) < 3]
member_df_nooutliers.shape

In [ ]:
fig, ax = plt.subplots()
sns.distplot(member_df_nooutliers.Duration, bins=50, ax=ax);

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(casual_df.Duration);

In [ ]:
casual_df_nooutliers = casual_df[np.abs(stats.zscore(casual_df.Duration)) < 3]

In [ ]:
fig, ax = plt.subplots()
sns.distplot(casual_df_nooutliers.Duration, bins=20, ax=ax);

In [ ]:
fig, ax = plt.subplots()
sns.distplot(member_df_nooutliers.Duration, bins=20, ax=ax, color='blue')
sns.distplot(casual_df_nooutliers.Duration, bins=20, ax=ax, color='green');

#### Get population statistics

In [ ]:
df.median()

Let's treat the whole dataset as our population.

In [ ]:
pop_mean = df.Duration.mean()
pop_std = df.Duration.std()
print(f'pop_mean is {pop_mean} \npop_std is {pop_std}')

In [ ]:
def one_sample_mean(population):
    sample = np.random.choice(population, size=200, replace=True)
    return sample.mean()

In [ ]:
one_sample_mean(df.Duration)

### When we take multiple samples from the distribution,and plot the means of each sample, the shape of the curve shifts

In [ ]:
d = [one_sample_mean(df.Duration) for i in range(1000)]
plt.hist(d, bins=50);

In [ ]:
def central_limit_theorem_plotter(distribution, sample_size, num_samples, color='blue'):
    sample_means = np.zeros(num_samples)
    for idx, num in enumerate(range(num_samples)):
        sample = np.random.choice(distribution, size=sample_size, replace=True)
        sample_means[idx] = sample.mean()
    sns.distplot(sample_means, bins=80, kde=True,  color=color)
    title = f'Sample Distribution n = {sample_size} and number of samples = {num_samples},\
    std error = {pop_std / num_samples}'
    print(f'mean = {sample_means.mean()}')
    plt.title(title)

### The number of samples drives the shape of the curve more than the sample size itself


In [ ]:
central_limit_theorem_plotter(df.Duration, 1000, 500);

### Larger sample size, Fewer samples

In [ ]:
central_limit_theorem_plotter(df.Duration, 5000, 50);

* What happens as we increase the sample size?
* How does the height of the distribution change? Why does it change?

In [ ]:
central_limit_theorem_plotter(member_df.Duration, 1000, 500, 'blue')
central_limit_theorem_plotter(casual_df.Duration, 1000, 500, 'green')